In [29]:
import cv2

###
### <center> Initialization of BigQuery </center>

In [30]:
def initialize_bigquery(project_id, dataset_id, table_id_file):

    import os
    from google.cloud import bigquery
    from datetime import datetime
    import threading

    # check if a table file exists, load it or make one
    if os.path.exists(table_id_file):
        with open(table_id_file, 'r') as file:
            table_id = file.read().strip()
    else:
        table_id = f'test1_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
        with open(table_id_file, 'w') as file:
            file.write(table_id)

    # create or load the bigquery table
    client = bigquery.Client(project=project_id)
    table = client.create_table(
        bigquery.Table(f"{project_id}.{dataset_id}.{table_id}", 
                       
        # this schema will likely change at some point, it is easily editable here
        schema=[
            bigquery.SchemaField("qr_data", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("timestamp", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("Position", "STRING", mode="NULLABLE")
        ]), 

        exists_ok=True)

    return client, table

###
### <center> Uploading Data to BigQuery <center>
##### <center> (for the threading process) </center>

In [31]:
def upload_data():

    import time
    import threading

    upload_interval = 10
    data_lock = threading.Lock()
    global batched_data

    while True:

        # we are upload every {upload_interval} seconds
        time.sleep(upload_interval)
        
        # the "data_lock" and "batched_data" serves to prevent the threading from imploding
        with data_lock:
            if not batched_data:
                continue
            
            data_to_insert = batched_data.copy()
            batched_data.clear()

        # prepare the data
        rows_to_insert = [
            {"qr_data": data, "timestamp": timestamp.isoformat(), "Position": position} 
            for data, timestamp, position in data_to_insert
        ]

        # upload the data
        errors = client.insert_rows_json(table, rows_to_insert)
        
        if errors:
            print(f"Encountered errors while inserting rows: {errors}")
        else:
            print(f"Data uploaded to {table.table_id} at {datetime.utcnow().isoformat()}")

###
### <center> Display Lines on Webcam for QR Codes</center>

In [32]:
def draw_lines(count, img, decoded_list, points, color):
    for code in decoded_list:
        count += 1
        img = cv2.polylines(img, points, True, color, 8)
    return count
def flip_y(y, mid):
    return (mid-y)+mid

###
### <center> The "Main" Program </center>

In [33]:
# batched data is defined here instead of inside the main function so
# that it can exist as a global variable used by main() and upload_data()
batched_data = []

In [ ]:
def main():

    import json
    import threading
    from datetime import datetime
    from pylibdmtx.pylibdmtx import decode as pyl_decode
    import numpy as np

    # we start with some initializations
    my_client, my_table = initialize_bigquery('finch-project-399922', 'finch_beta_table', 'table_id.txt')    
    display_select = False
    data_lock = threading.Lock()
    global batched_data

    # then we open a thread for constant bigquery uploads
    upload_thread = threading.Thread(target=upload_data, daemon=True)
    upload_thread.start()

    # turn on the webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Could not open webcam")
        return

    # this while loop constantly reads data frame the webcam
    # specifically looking to read and process qr codes
    while True:

        ret, frame = cap.read()
        if not ret: # ret is a boolean telling us if we successfully read a frame
            break

        # then we read the data 
        # shrink - divides img quality; thresh - detection; inc both improves speed lowers detection rate
        pyl_decoded  = pyl_decode(frame, shrink=1, threshold=12)
        if pyl_decoded:
            for dm_data in pyl_decoded:
                print("DM Data: ", dm_data.data)
                with data_lock:
                    batched_data.append((dm_data, datetime.utcnow(), None))

        # adding options to webcam display frame
        frame = cv2.putText(frame, "a - display", (540,450), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
        frame = cv2.putText(frame, "q - quit", (540,470), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)

        # toggle code frame lines on "a", needs to be before display or no lines will be drawn
        if cv2.waitKey(1) & 0xFF == ord('a'):
            display_select = not display_select

        boxes_json = {}

        if display_select:
            img_mid = frame.shape[0]/2
            points_pyl = []
            pyl_count = 0
            for i in range(0, len(pyl_decoded)):
                #need to format to cv2 polygon to draw
                left, top, width, height = pyl_decoded[i].rect
                top_left = [left, flip_y(top, img_mid)]
                btm_left = [left, flip_y(top + height, img_mid)]
                top_right = [left + width, flip_y(top, img_mid)]
                btm_right = [left + width, flip_y(top + height, img_mid)]
                points_pyl += [np.array([top_left, btm_left, btm_right, top_right], np.int32)]
                pyl_count = draw_lines(pyl_count, frame, pyl_decoded, points_pyl, (0, 0, 255))


                #this adjusts the format of points for input into the distance algorithm
                points = [top_left, btm_left, btm_right, top_right]
                key = f"qr{i}"
                boxes_json[key] = [tuple(point) for point in points_pyl]
                

        # webcam display frame
        cv2.imshow("QR Code Scanner", frame)

        # break on "q"
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [35]:
main()

DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'3'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'3'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'3'
DM Data:  b'4'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'3'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'4'
DM Data:  b'3'
DM Data:  b'5'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'3'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'4'
DM Data:  b'5'


Exception in thread Thread-8 (upload_data):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\carme\AppData\Local\Temp\ipykernel_1796\3819515573.py", line 30, in upload_data
NameError: name 'client' is not defined


DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'3'
DM Data:  b'4'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'5'
DM Data:  b'3'
DM Data:  b'4'
DM Data:  b'<'
DM Data:  b'3'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'3'
DM Data:  b'5'
DM Data:  b'4'
DM Data:  b'53'
DM Data:  b'1'
DM Data:  b'4'
DM Data:  b'2'
DM Data:  b'3'
DM Data:  b'5'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'3'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'3'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'1'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'3'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'3'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'00F'
DM Data:  b'5'
DM Data:  b'3'
DM Data:  b'1'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'3'
DM Data:  b'5'
DM Data:  b'1'
DM Data:  b'3'
DM Data:  b'2'
DM Data:  b'4'
DM Data:  b'"6568'
